In [1]:
import nltk
from nltk import CFG
from nltk.tree import Tree


In [4]:

# Define a context-free grammar
cfg_string = """
S -> NP VP
VP -> V NP | V NP PP
PP -> P NP
V -> 'saw' | 'ate' | 'walked'
NP -> 'John' | 'Mary' | 'Bob' | Det N | Det N PP
Det -> 'a' | 'an' | 'the' | 'my'
N -> 'man' | 'dog' | 'cat' | 'telescope' | 'park'
P -> 'in' | 'on' | 'by' | 'with'
"""

# Parse the CFG string
grammar = CFG.fromstring(cfg_string)

# Implement the CKY algorithm
def cky_parse(sentence, grammar):
    words = sentence.split()
    n = len(words)
    table = [[set() for _ in range(n+1)] for _ in range(n)]
    backpointers = [[{} for _ in range(n+1)] for _ in range(n)]

    for j in range(1, n+1):
        word = words[j-1]
        for prod in grammar.productions(rhs=word):
            table[j-1][j].add(prod.lhs())
            backpointers[j-1][j][prod.lhs()] = (word,)

        for i in range(j-2, -1, -1):
            for k in range(i+1, j):
                for prod in grammar.productions():
                    if len(prod.rhs()) == 2:
                        B, C = prod.rhs()
                        if B in table[i][k] and C in table[k][j]:
                            table[i][j].add(prod.lhs())
                            backpointers[i][j][prod.lhs()] = (k, B, C)

    return table, backpointers

def construct_tree(backpointers, i, j, symbol):
    if j - i == 1:
        return symbol
    k, B, C = backpointers[i][j][symbol]
    left_subtree = construct_tree(backpointers, i, k, B)
    right_subtree = construct_tree(backpointers, k, j, C)
    return Tree(symbol, [left_subtree, right_subtree])

# Parse a given sentence and display the dependency tree
def parse_sentence(sentence, grammar):
    table, backpointers = cky_parse(sentence, grammar)
    if grammar.start() in table[0][len(sentence.split())]:
        parse_tree = construct_tree(backpointers, 0, len(sentence.split()), grammar.start())
        parse_tree.pretty_print()
    else:
        print("No valid parse tree found.")

# Example usage
sentence = "John saw Mary with telescope"
parse_sentence(sentence, grammar)


No valid parse tree found.
